In [1]:
import pandas as pd
import cv2
import sys
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

# Add the directory containing your module to the Python path
sys.path.append('/app')

from utils import load_metadata, map_paths, split_data, generate_features, results_to_dataframe, save_features, process_test_set
from augmentations import get_augmentations
from radiomics import featureextractor
import albumentations as A

# Load metadata
data_dir = "/app/data"
image_dirs = [os.path.join(data_dir, 'HAM10000_images_part_1'), os.path.join(data_dir, 'HAM10000_images_part_2')]
mask_dir = os.path.join(data_dir, 'HAM10000_segmentations_lesion_tschandl')
metadata_file = os.path.join(data_dir, 'HAM10000_metadata')  # Corrected the path to the CSV file

df = load_metadata(metadata_file)
df = map_paths(df, image_dirs, mask_dir)

# Split data into train and validation
#df_train, df_val = split_data(df)

# Define augmentations
augmentations = get_augmentations()

# Initialize the radiomics feature extractor
extractor = featureextractor.RadiomicsFeatureExtractor('/app/config.yaml')

# Choose mode: 'grayscale' or 'rgb'
mode = 'rgb'

# # Generate features for training set with augmentation
# print("Generating features for training set...")
# train_results = generate_features(df, extractor, augmentations, mode)
# df_train_features = results_to_dataframe(train_results, df)
# save_features(df_train_features, '/app/data/train_validation_features.csv')

# Define resize transform for test and validation set
resize_transform = A.Compose([A.Resize(height=224, width=224)], is_check_shapes=False)

# Process the new test dataset
test_image_dir = "/app/data/ISIC2018_Task3_Test_Images"
test_mask_dir = "/app/data/ISIC2018_Task3_Test_GroundTruth"
test_metadata_file = "/app/data/ISIC2018_Task3_Test_GroundTruth.csv"
output_test_features = '/app/data/test_features.csv'

print("Processing the new test dataset...")
process_test_set(test_image_dir, test_mask_dir, test_metadata_file, extractor, output_test_features, resize_transform, mode=mode)

Processing the new test dataset...


Processing Test Set: 100%|██████████| 1511/1511 [02:39<00:00,  9.46it/s]


In [2]:
df = pd.read_csv('/app/data/test_features.csv')
df

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image_path,mask_path,...,original_gldm_GrayLevelNonUniformity_b,original_gldm_GrayLevelVariance_b,original_gldm_HighGrayLevelEmphasis_b,original_gldm_LargeDependenceEmphasis_b,original_gldm_LargeDependenceHighGrayLevelEmphasis_b,original_gldm_LargeDependenceLowGrayLevelEmphasis_b,original_gldm_LowGrayLevelEmphasis_b,original_gldm_SmallDependenceEmphasis_b,original_gldm_SmallDependenceHighGrayLevelEmphasis_b,original_gldm_SmallDependenceLowGrayLevelEmphasis_b
0,HAMTEST_0000000,ISIC_0034524,nv,follow_up,40.0,female,back,vidir_molemax,/app/data/ISIC2018_Task3_Test_Images/ISIC_0034...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,3699.994877,0.967544,15.398940,59.709985,931.723735,6.162142,0.111446,0.034656,0.540776,0.004462
1,HAMTEST_0000001,ISIC_0034525,nv,histo,70.0,male,abdomen,rosendahl,/app/data/ISIC2018_Task3_Test_Images/ISIC_0034...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,2549.872746,1.649065,13.233240,59.675827,786.439961,8.762261,0.156433,0.030544,0.421441,0.006004
2,HAMTEST_0000002,ISIC_0034526,bkl,histo,70.0,male,back,rosendahl,/app/data/ISIC2018_Task3_Test_Images/ISIC_0034...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,3298.905597,1.358745,25.356649,53.206195,1371.525891,2.854635,0.055877,0.037483,0.923273,0.002427
3,HAMTEST_0000003,ISIC_0034527,nv,histo,35.0,male,trunk,vienna_dias,/app/data/ISIC2018_Task3_Test_Images/ISIC_0034...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,15807.435719,1.074727,19.913856,57.010869,1143.973470,3.458497,0.061961,0.037841,0.765408,0.002549
4,HAMTEST_0000004,ISIC_0034528,nv,follow_up,75.0,female,trunk,vidir_molemax,/app/data/ISIC2018_Task3_Test_Images/ISIC_0034...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,6433.907824,0.868290,12.454056,51.565071,640.501826,5.359492,0.109401,0.049570,0.639500,0.006342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,HAMTEST_0001219,ISIC_0036060,bkl,consensus,NaN,unknown,unknown,external,/app/data/ISIC2018_Task3_Test_Images/ISIC_0036...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,3844.000683,2.656464,32.019806,37.100533,1164.640213,1.836290,0.050533,0.082034,2.679813,0.004588
1507,HAMTEST_0001220,ISIC_0036061,nv,consensus,NaN,unknown,unknown,external,/app/data/ISIC2018_Task3_Test_Images/ISIC_0036...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,2964.620504,2.835302,27.136519,51.737909,1500.025833,3.347452,0.075787,0.043220,0.992575,0.005467
1508,HAMTEST_0001107,ISIC_0036062,akiec,histo,70.0,female,face,vidir_modern,/app/data/ISIC2018_Task3_Test_Images/ISIC_0036...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,2054.346633,0.860888,16.805780,50.313774,878.277395,3.675999,0.078858,0.037889,0.653947,0.003398
1509,HAMTEST_0001221,ISIC_0036063,bkl,consensus,NaN,unknown,unknown,external,/app/data/ISIC2018_Task3_Test_Images/ISIC_0036...,/app/data/ISIC2018_Task3_Test_GroundTruth/ISIC...,...,7418.010274,2.854810,23.106855,54.662499,1229.795495,4.469967,0.078590,0.037039,0.859100,0.002990


In [3]:
df = pd.read_csv('/app/data/val_features.csv')
df

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,mask_id,cell_type,...,original_gldm_GrayLevelNonUniformity_b,original_gldm_GrayLevelVariance_b,original_gldm_HighGrayLevelEmphasis_b,original_gldm_LargeDependenceEmphasis_b,original_gldm_LargeDependenceHighGrayLevelEmphasis_b,original_gldm_LargeDependenceLowGrayLevelEmphasis_b,original_gldm_LowGrayLevelEmphasis_b,original_gldm_SmallDependenceEmphasis_b,original_gldm_SmallDependenceHighGrayLevelEmphasis_b,original_gldm_SmallDependenceLowGrayLevelEmphasis_b
0,HAM_0005575,ISIC_0029805,bcc,histo,85.0,male,back,rosendahl,ISIC_0029805_segmentation,Basal cell carcinoma,...,7124.910905,1.091443,28.487057,63.370456,1848.346099,2.766553,0.046964,0.025495,0.692016,0.001455
1,HAM_0003279,ISIC_0030329,bkl,histo,70.0,female,lower extremity,rosendahl,ISIC_0030329_segmentation,Benign keratosis-like lesions,...,11190.570910,0.687243,26.450973,63.836515,1708.440047,2.675757,0.043108,0.027613,0.687606,0.001478
2,HAM_0005385,ISIC_0028754,nv,follow_up,45.0,male,abdomen,vidir_molemax,ISIC_0028754_segmentation,Melanocytic nevi,...,6139.646863,1.430008,9.048881,61.995425,472.234568,12.396310,0.198501,0.031264,0.312325,0.010053
3,HAM_0003503,ISIC_0027555,nv,follow_up,60.0,female,lower extremity,vidir_molemax,ISIC_0027555_segmentation,Melanocytic nevi,...,3977.176229,1.789100,12.846859,64.172876,746.946043,9.671108,0.139346,0.024854,0.350641,0.003092
4,HAM_0002020,ISIC_0032301,nv,follow_up,50.0,male,trunk,vidir_molemax,ISIC_0032301_segmentation,Melanocytic nevi,...,5671.532570,0.717973,9.700610,61.033231,575.676946,10.920238,0.176767,0.033901,0.341996,0.006617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,HAM_0004381,ISIC_0027426,nv,follow_up,45.0,female,abdomen,vidir_molemax,ISIC_0027426_segmentation,Melanocytic nevi,...,3579.014225,1.222979,11.740683,43.071550,479.410100,6.146637,0.140004,0.065733,0.913835,0.010646
1099,HAM_0006877,ISIC_0028227,mel,histo,55.0,male,upper extremity,rosendahl,ISIC_0028227_segmentation,Melanoma,...,3000.418371,3.600932,32.645793,49.556831,1685.676959,2.661129,0.055894,0.038855,1.184027,0.002580
1100,HAM_0006920,ISIC_0025432,nv,follow_up,50.0,female,trunk,vidir_molemax,ISIC_0025432_segmentation,Melanocytic nevi,...,8923.137162,0.883429,14.388436,53.898311,759.856110,4.925950,0.093002,0.047209,0.743492,0.004743
1101,HAM_0000932,ISIC_0026288,nv,follow_up,30.0,female,trunk,vidir_molemax,ISIC_0026288_segmentation,Melanocytic nevi,...,5837.678552,0.530330,18.380035,63.683587,1173.188419,4.011517,0.063693,0.030012,0.554576,0.002047


In [4]:
df = pd.read_csv('/app/data/train_features.csv')
df

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,mask_id,cell_type,...,original_gldm_GrayLevelNonUniformity_b,original_gldm_GrayLevelVariance_b,original_gldm_HighGrayLevelEmphasis_b,original_gldm_LargeDependenceEmphasis_b,original_gldm_LargeDependenceHighGrayLevelEmphasis_b,original_gldm_LargeDependenceLowGrayLevelEmphasis_b,original_gldm_LowGrayLevelEmphasis_b,original_gldm_SmallDependenceEmphasis_b,original_gldm_SmallDependenceHighGrayLevelEmphasis_b,original_gldm_SmallDependenceLowGrayLevelEmphasis_b
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0027419_segmentation,Benign keratosis-like lesions,...,4203.698479,2.146759,41.089061,43.510118,1885.614924,1.365962,0.033739,0.051098,1.949651,0.001951
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0025030_segmentation,Benign keratosis-like lesions,...,1468.857634,2.373909,31.744429,34.632497,1192.516618,1.771078,0.056234,0.071468,2.040454,0.005138
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0026769_segmentation,Benign keratosis-like lesions,...,4827.077116,1.079174,29.962584,51.969725,1587.792014,2.017942,0.041168,0.040376,1.212471,0.001828
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0025661_segmentation,Benign keratosis-like lesions,...,1344.831858,1.198353,26.950366,40.183532,1095.317045,1.833344,0.048847,0.059647,1.732144,0.003086
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,ISIC_0031633_segmentation,Benign keratosis-like lesions,...,4851.997838,2.046051,29.545957,46.183026,1337.747878,2.227432,0.048916,0.052254,1.652825,0.002692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8907,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,ISIC_0033084_segmentation,Actinic keratoses,...,7189.918952,2.083367,32.494624,47.884647,1622.012773,1.908658,0.044810,0.061964,1.897923,0.003579
8908,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,ISIC_0033550_segmentation,Actinic keratoses,...,2903.918567,1.694708,25.707409,44.416195,1186.592454,2.268788,0.056348,0.068386,1.736945,0.004654
8909,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,ISIC_0033536_segmentation,Actinic keratoses,...,3811.129338,2.145657,26.296300,43.811673,1206.020219,2.428819,0.064543,0.071575,1.826588,0.006026
8910,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern,ISIC_0032854_segmentation,Actinic keratoses,...,10591.114250,0.815401,17.693281,57.855099,1063.062233,3.957963,0.074582,0.032209,0.553124,0.002932
